In [1]:
#adapted from @bact at https://colab.research.google.com/drive/1hdtmwTXHLrqNmDhDqHnTQGpDVy1aJc4t
import json
import pandas as pd
import numpy as np
import re
import pycrfsuite
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from ast import literal_eval
from tqdm import tqdm
pd.set_option('display.max_rows', 10)
warnings.filterwarnings('ignore')
!cd data/fake_review; unzip fake_review.zip

Archive:  fake_review.zip
  inflating: fake_review_sent_segment.csv  


In [2]:
fake_review = pd.read_csv('data/fake_review/fake_review_sent_segment.csv')

In [3]:
fake_review

,review_id,th_segment_tokenize,correct,len_sent_segment
0,1-333090,['สินค้าคุณภาพดีในราคาที่เหมาะสม'],1.0,1
1,1-57783,['พวกเขาไม่เกิน 10 ขวด'],0.0,1
2,1-319174,['ถ้าคุณชอบนิยายหลังเลิกเรียนด้วยการบิดนี่คือม...,0.0,1
3,1-425331,['รู้สึกกังวลเล็กน้อยเมื่อซื้อเนื่องจากรีวิวบา...,0.0,1
4,1-134583,['ฉันได้ยินจากญาติที่อ่านหนังสือเล่มนี้ว่าพ่อแ...,1.0,1
...,...,...,...,...
217478,2-32438,['ฉันลอง Ipod ของเพื่อนและรู้สึกตื่นเต้นมากที่...,0.0,7
217479,2-36181,['นี่เป็นครั้งที่สามที่ฉันไปแทนที่กล้องรุ่นเก่...,0.0,8
217480,2-22475,['หนังสือไม่สามารถอ่านได้เนื่องจากมีหน้ากระดาษ...,1.0,8
217481,2-33909,['ฉันมีกล้องดิจิตอล Canon Powershot SX10 IS ให...,0.0,9


In [4]:
fake_review['th_segment_tokenize'] = fake_review['th_segment_tokenize'].apply(literal_eval)
fake_review['th_segment_tokenize'] = fake_review['th_segment_tokenize'].map(lambda x: ' |'.join(x))
fake_review['th_segment_tokenize'] = fake_review['th_segment_tokenize'] + ' '
fake_review

,review_id,th_segment_tokenize,correct,len_sent_segment
0,1-333090,สินค้าคุณภาพดีในราคาที่เหมาะสม,1.0,1
1,1-57783,พวกเขาไม่เกิน 10 ขวด,0.0,1
2,1-319174,ถ้าคุณชอบนิยายหลังเลิกเรียนด้วยการบิดนี่คือมัน,0.0,1
3,1-425331,รู้สึกกังวลเล็กน้อยเมื่อซื้อเนื่องจากรีวิวบางอ...,0.0,1
4,1-134583,ฉันได้ยินจากญาติที่อ่านหนังสือเล่มนี้ว่าพ่อแม่...,1.0,1
...,...,...,...,...
217478,2-32438,ฉันลอง Ipod ของเพื่อนและรู้สึกตื่นเต้นมากที่จะ...,0.0,7
217479,2-36181,นี่เป็นครั้งที่สามที่ฉันไปแทนที่กล้องรุ่นเก่าก...,0.0,8
217480,2-22475,หนังสือไม่สามารถอ่านได้เนื่องจากมีหน้ากระดาษขา...,1.0,8
217481,2-33909,ฉันมีกล้องดิจิตอล Canon Powershot SX10 IS ใหม่...,0.0,9


In [5]:
all_sentences = []
for i, row in fake_review.iterrows():
    all_sentences.append(row['th_segment_tokenize'])

In [6]:
all_sentences[:10]

['สินค้าคุณภาพดีในราคาที่เหมาะสม ',
 'พวกเขาไม่เกิน 10 ขวด ',
 'ถ้าคุณชอบนิยายหลังเลิกเรียนด้วยการบิดนี่คือมัน ',
 'รู้สึกกังวลเล็กน้อยเมื่อซื้อเนื่องจากรีวิวบางอย่างเป็นลบ ',
 'ฉันได้ยินจากญาติที่อ่านหนังสือเล่มนี้ว่าพ่อแม่ของพวกเขารักมันมากแค่ไหน ',
 'เครื่องพิมพ์ไร้สาย Epson WorkForce นั้นยอดเยี่ยมกว่าเครื่องพิมพ์บราเดอร์หรือแคนนอนของคุณ ',
 'การเขียนไหลเวียนได้ดีตลอดทั้งเล่ม |อย่างไรก็ตามมีบางส่วนที่รู้สึกไม่สมจริง ',
 'รายการนี้ยอดเยี่ยมมาก |มันทำให้เสื้อสุนัขแห้ง |แต่มันก็รั่วไหล ',
 'นี่อาจจะดีพอที่จะอยู่ได้ตลอดชีวิตหรือสำหรับเด็กอย่างน้อยหนึ่งคนถ้าคุณไม่คลั่งไคล้มัน ',
 'มันเป็นแท็บเล็ตที่ค่อนข้างดี แต่ดูเหมือนว่าจะช้าเมื่อเทียบกับแท็บเล็ตอื่นที่ฉันซื้อ |ยังมีความสุขโดยรวมจนถึงตอนนี้ ']

In [24]:
len(all_sentences)

217483

In [7]:
all_tuples = []
for i in range(len(all_sentences)):
    tuples = []
    for s in all_sentences[i].split('|'):
        s_lst = word_tokenize(s)
        for j in range(len(s_lst)):
            lab = 'E' if j==len(s_lst)-1 else 'I'
            tuples.append((s_lst[j],lab))
    all_tuples.append(tuples)

In [8]:
len(all_tuples)

217483

In [26]:
all_tuples[:3]

[[('สินค้า', 'I'),
  ('คุณภาพดี', 'I'),
  ('ใน', 'I'),
  ('รา', 'I'),
  ('คาที่', 'I'),
  ('เหมาะสม', 'I'),
  (' ', 'E')],
 [('พวกเขา', 'I'),
  ('ไม่', 'I'),
  ('เกิน', 'I'),
  (' ', 'I'),
  ('10', 'I'),
  (' ', 'I'),
  ('ขวด', 'I'),
  (' ', 'E')],
 [('ถ้า', 'I'),
  ('คุณ', 'I'),
  ('ชอบ', 'I'),
  ('นิยาย', 'I'),
  ('หลัง', 'I'),
  ('เลิกเรียน', 'I'),
  ('ด้วย', 'I'),
  ('การ', 'I'),
  ('บิด', 'I'),
  ('นี่', 'I'),
  ('คือ', 'I'),
  ('มัน', 'I'),
  (' ', 'E')]]

In [21]:
word_tokenize('ผมมากินมะละกอ ')

['ผม', 'มา', 'กิน', 'มะละกอ', ' ']

In [10]:
enders = ["ครับ","ค่ะ","คะ","นะคะ","นะ","จ้ะ","จ้า","จ๋า","ฮะ", #ending honorifics
          #enders
          "ๆ","ได้","แล้ว","ด้วย","เลย","มาก","น้อย","กัน","เช่นกัน","เท่านั้น",
          "อยู่","ลง","ขึ้น","มา","ไป","ไว้","เอง","อีก","ใหม่","จริงๆ",
          "บ้าง","หมด","ทีเดียว","เดียว",
          #demonstratives
          "นั้น","นี้","เหล่านี้","เหล่านั้น",
          #questions
          "อย่างไร","ยังไง","หรือไม่","มั้ย","ไหน","อะไร","ทำไม","เมื่อไหร่"]
starters = ["ผม","ฉัน","ดิฉัน","ชั้น","คุณ","มัน","เขา","เค้า",
            "เธอ","เรา","พวกเรา","พวกเขา", #pronouns
            #connectors
            "และ","หรือ","แต่","เมื่อ","ถ้า","ใน",
            "ด้วย","เพราะ","เนื่องจาก","ซึ่ง","ไม่",
            "ตอนนี้","ทีนี้","ดังนั้น","เพราะฉะนั้น","ฉะนั้น",
            "ตั้งแต่","ในที่สุด",
            #demonstratives
            "นั้น","นี้","เหล่านี้","เหล่านั้น"]

def extract_features(doc, window=2, max_n_gram=3):
    doc_features = []
    #paddings for word and POS
#     doc_pos = ['xxpad' for i in range(window)] + \
#         [p for (w,p) in pos_tag(doc,engine='artagger', corpus='orchid')] + ['xxpad' for i in range(window)]
    doc = ['xxpad' for i in range(window)] + doc + ['xxpad' for i in range(window)]
    doc_ender = []
    doc_starter = []
    #add enders
    for i in range(len(doc)):
        if doc[i] in enders:
            doc_ender.append('ender')
        else:
            doc_ender.append('normal')
    #add starters
    for i in range(len(doc)):
        if doc[i] in starters:
            doc_starter.append('starter')
        else:
            doc_starter.append('normal')
    #for each word
    for i in range(window, len(doc)-window):
        #bias term
        word_features = ['bias'] 
        
        #ngram features
        for n_gram in range(1, min(max_n_gram+1,2+window*2)):
            for j in range(i-window,i+window+2-n_gram):
                feature_position = f'{n_gram}_{j-i}_{j-i+n_gram}'
                word_ = f'{"|".join(doc[j:(j+n_gram)])}'
                word_features += [f'word_{feature_position}={word_}']
#                 pos_ =f'{"|".join(doc_pos[j:(j+n_gram)])}'
#                 word_features += [f'pos_{feature_position}={pos_}']
                ender_ =  f'{"|".join(doc_ender[j:(j+n_gram)])}'
                word_features += [f'ender_{feature_position}={ender_}']
                starter_ =  f'{"|".join(doc_starter[j:(j+n_gram)])}'
                word_features += [f'starter_{feature_position}={starter_}']
        
#         #number of verbs to the left and right
#         nb_verbs_left = 0
#         for l in range(i)[::-1]:
#             if doc[l]=='<space>': break
#             if doc_pos[l]=='VACT': nb_verbs_left+=1
#         nb_verbs_right = 0
#         for r in range(i+1,len(doc)):
#             if doc[r]=='<space>': break
#             if doc_pos[r]=='VACT': nb_verbs_right+=1
#         word_features += [f'nb_verbs_left={nb_verbs_left}',f'nb_verbs_right={nb_verbs_right}']
        
        #append to feature per word
        doc_features.append(word_features)
    return doc_features

In [11]:
#target
y = [[l for (w,l) in t] for t in all_tuples]
#features
x_pre = [[w for (w,l) in t] for t in all_tuples]
x = [extract_features(x_, window=2, max_n_gram = 3) for x_ in x_pre]

In [12]:
len(x),len(y)

(217483, 217483)

In [13]:
# Split train and test set at 80/20 proportion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1412)

In [15]:
%%time
# Train model
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in tqdm(zip(x_train, y_train)):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1,
    'c2': 0,
    'max_iterations': 1000,
    'feature.possible_transitions': True,
})

trainer.train('models/fake-review-crf.model')

173986it [40:40, 71.31it/s]


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 13140527
Seconds required: 277.548

L-BFGS optimization
c1: 1.000000
c2: 0.000000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 4899674.737881
Feature norm: 1.000000
Error norm: 4981632.084108
Active features: 2541662
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 58.797

***** Iteration #2 *****
Loss: 4200121.105395
Feature norm: 0.864793
Error norm: 4771532.592226
Active features: 2010507
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 17.630

***** Iteration #3 *****
Loss: 2570110.578790
Feature norm: 0.542342
Error norm: 1605288.363564
Active features: 993279
Line search trials: 4
Line search step: 0.12

In [ ]:
# ***** Iteration #363 *****
# Loss: 135929.594532
# Feature norm: 120.879634
# Error norm: 178.644938
# Active features: 60879
# Line search trials: 1
# Line search step: 1.000000
# Seconds required for this iteration: 17.484

# L-BFGS terminated with the stopping criteria
# Total seconds required for training: 6656.525

# Storing the model
# Number of active features: 60879 (13140527)
# Number of active attributes: 33199 (12892063)
# Number of active labels: 2 (2)
# Writing labels
# Writing attributes
# Writing feature references for transitions
# Writing feature references for attributes
# Seconds required: 3.433

In [18]:
# Predict (using test set)
tagger = pycrfsuite.Tagger()
tagger.open('models/fake-review-crf.model')
# y_pred = [tagger.tag(xseq) for xseq in x_test]
y_pred = []
for xseq in tqdm(x_test, total=len(x_test)):
    y_pred.append(tagger.tag(xseq))

100%|██████████| 43497/43497 [03:20<00:00, 217.11it/s]


In [19]:
# Evaluate at word-level
labels = {'E': 0, "I": 1} # classification_report() needs values in 0s and 1s
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["E", "I"]))

              precision    recall  f1-score   support

           E       0.98      0.96      0.97    194361
           I       1.00      1.00      1.00   2597236

    accuracy                           1.00   2791597
   macro avg       0.99      0.98      0.98   2791597
weighted avg       1.00      1.00      1.00   2791597



In [ ]:
#               precision    recall  f1-score   support

#            E       0.98      0.96      0.97    194361
#            I       1.00      1.00      1.00   2597236

#     accuracy                           1.00   2791597
#    macro avg       0.99      0.98      0.98   2791597
# weighted avg       1.00      1.00      1.00   2791597